In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install scenedetect[opencv]
!cd /content/drive/MyDrive/Dense-Video-Captioning/

!ls /content/drive/MyDrive/Dense-Video-Captioning/*.py

!cat /content/drive/MyDrive/Dense-Video-Captioning/config.py

import sys
sys.path.append('/content/drive/MyDrive/Dense-Video-Captioning/')
import os
os.chdir('/content/drive/MyDrive')
import config
from scenedetect import VideoManager, SceneManager, StatsManager
from scenedetect.detectors import ContentDetector
from scenedetect.scene_manager import save_images, write_scene_list_html
from google.colab.patches import cv2_imshow
import math

     |████████████████████████████████| 98 kB 4.1 MB/s 
/content/drive/MyDrive/Dense-Video-Captioning/config.py
/content/drive/MyDrive/Dense-Video-Captioning/extract_features.py
train_path = "/content/drive/MyDrive/Dense-Video-Captioning/data/training_data"
test_path = "/content/drive/MyDrive/Dense-Video-Captioning/data/testing_data/"
batch_size = 320
learning_rate = 0.0007
epochs = 150
latent_dim =256
num_encoder_tokens = 2560
num_decoder_tokens = 1500
time_steps_encoder = 20
max_probability = -1
save_model_path = 'model_stacked'
validation_split = 0.15
max_length = 10
search_type = 'beam'


In [3]:
import functools
import operator
import os
import time
import json
import joblib
import numpy as np
from keras.layers import Input, LSTM, Dense
from keras.models import Model, load_model
import config
import nltk
nltk.download('all')
from nltk.tokenize import word_tokenize
import math
import extract_features

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

In [4]:
import config
class VideoDescriptionInference(object):
    """
            Initialize the parameters for the model
            """
    def __init__(self, config):
        self.latent_dim = config.latent_dim
        print(self.latent_dim)
        self.num_encoder_tokens = config.num_encoder_tokens
        self.num_decoder_tokens = config.num_decoder_tokens
        self.time_steps_encoder = config.time_steps_encoder
        self.max_probability = config.max_probability

        # models
        self.encoder_model = None
        self.decoder_model = None
        self.inf_encoder_model = None
        self.inf_decoder_model = None
        self.save_model_path = config.save_model_path
        self.test_path = config.test_path
        self.search_type = config.search_type
        self.tokenizer = None

    def load_inference_models(self):
        # load tokenizer

        with open("/content/drive/MyDrive/Dense-Video-Captioning/model_stacked/tokenizer1500", 'rb') as file:
            self.tokenizer = joblib.load(file)
        print("tokenixer")
        # inference encoder model
        self.inf_encoder_model = load_model("/content/drive/MyDrive/Dense-Video-Captioning/model_stacked/encoder_model.h5")
        print("encoder")
        """decoder_lstm1 = LSTM(latent_dim*2, return_sequences=True, return_state=True, name='decoder_lstm')
        d_outputs, _, _ = decoder_lstm1(decoder_inputs, initial_state=encoder_states)
        decoder_lstm2 = LSTM(latent_dim*2, return_sequences=True, return_state=True, name='decoder_lstm2')
        final, dh2, dc2 =decoder_lstm2(d_outputs,initial_state=[state_h, state_c])
        decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_relu')
        decoder_outputs = decoder_dense(final)
        """
        # inference decoder model
        decoder_inputs = Input(shape=(None, self.num_decoder_tokens))
        decoder_dense = Dense(self.num_decoder_tokens, activation='softmax')
        decoder_lstm1 = LSTM(self.latent_dim*2, return_sequences=True, return_state=True)
        decoder_state_input_h = Input(shape=(self.latent_dim*2,))
        decoder_state_input_c = Input(shape=(self.latent_dim*2,))
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder_outputs1, state_h1, state_c1 = decoder_lstm1(decoder_inputs, initial_state=decoder_states_inputs)

        decoder_lstm2 = LSTM(self.latent_dim*2, return_sequences=True, return_state=True)
        decoder_outputs, state_h, state_c =decoder_lstm2(decoder_outputs1,initial_state=[state_h1, state_c1])

        decoder_states = [state_h, state_c]
        decoder_outputs = decoder_dense(decoder_outputs)
        self.inf_decoder_model = Model(
            [decoder_inputs] + decoder_states_inputs,
            [decoder_outputs] + decoder_states)
        self.inf_decoder_model.load_weights("/content/drive/MyDrive/Dense-Video-Captioning/model_stacked/decoder_model_weights.h5")
        print("decoder")

    def greedy_search(self, f):
        """

                :param f: the loaded numpy array after creating videos to frames and extracting features
                :return: the final sentence which has been predicted greedily
                """
        inv_map = self.index_to_word()
        states_value = self.inf_encoder_model.predict(f.reshape(-1, 20,2560))
        target_seq = np.zeros((1, 1, 1500))
        sentence = ''
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        for i in range(15):
            output_tokens, h, c = self.inf_decoder_model.predict([target_seq] + states_value)
            states_value = [h, c]
            output_tokens = output_tokens.reshape(self.num_decoder_tokens)
            y_hat = np.argmax(output_tokens)
            if y_hat == 0:
                continue
            if inv_map[y_hat] is None:
                break
            else:
                sentence = sentence + inv_map[y_hat] + ' '
                target_seq = np.zeros((1, 1, 1500))
                target_seq[0, 0, y_hat] = 1
        return ' '.join(sentence.split()[:-1])

    def decode_sequence2bs(self, input_seq):
        states_value = self.inf_encoder_model.predict(input_seq)
        target_seq = np.zeros((1, 1, self.num_decoder_tokens))
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        self.beam_search(target_seq, states_value, [], [], 0)
        return decode_seq

    def beam_search(self, target_seq, states_value, prob, path, lens):
        """

                :param target_seq: the array that is fed into the model to predict the next word
                :param states_value: previous state that is fed into the lstm cell
                :param prob: probability of predicting a word
                :param path: list of words from each sentence
                :param lens: number of words
                :return: final sentence
                """
        global decode_seq
        node = 2
        output_tokens, h, c = self.inf_decoder_model.predict(
            [target_seq] + states_value)
        output_tokens = output_tokens.reshape(self.num_decoder_tokens)
        sampled_token_index = output_tokens.argsort()[-node:][::-1]
        states_value = [h, c]
        for i in range(node):
            if sampled_token_index[i] == 0:
                sampled_char = ''
            else:
                sampled_char = list(self.tokenizer.word_index.keys())[
                    list(self.tokenizer.word_index.values()).index(sampled_token_index[i])]
            MAX_LEN = 12
            if sampled_char != 'eos' and lens <= MAX_LEN:
                p = output_tokens[sampled_token_index[i]]
                if sampled_char == '':
                    p = 1
                prob_new = list(prob)
                prob_new.append(p)
                path_new = list(path)
                path_new.append(sampled_char)
                target_seq = np.zeros((1, 1, self.num_decoder_tokens))
                target_seq[0, 0, sampled_token_index[i]] = 1.
                self.beam_search(target_seq, states_value, prob_new, path_new, lens + 1)
            else:
                p = output_tokens[sampled_token_index[i]]
                prob_new = list(prob)
                prob_new.append(p)
                p = functools.reduce(operator.mul, prob_new, 1)
                if p > self.max_probability:
                    decode_seq = path
                    self.max_probability = p

    def decoded_sentence_tuning(self, decoded_sentence):
        decode_str = []
        filter_string = ['bos', 'eos']
        uni_gram = {}
        last_string = ""
        for idx2, c in enumerate(decoded_sentence):
            if c in uni_gram:
                uni_gram[c] += 1
            else:
                uni_gram[c] = 1
            if last_string == c and idx2 > 0:
                continue
            if c in filter_string:
                continue
            if len(c) > 0:
                decode_str.append(c)
            if idx2 > 0:
                last_string = c
        return decode_str

    def index_to_word(self):
        # inverts word tokenizer
        index_to_word = {value: key for key, value in self.tokenizer.word_index.items()}
        return index_to_word

    def get_test_data(self):
        """
        loads all the numpy files
        :return: two lists containing all the video arrays and the video Id
        """
        X_test = []
        X_test_filename = []
        file_list = os.listdir(os.path.join(self.test_path, 'video'))
        for index in range(len(file_list)):
            filename= file_list[index]
            scenes = extract_features.proposal_module(filename)
            scenes= math.floor(len(scenes)/2)
            featList=[]
            for i in range(0, scenes):
              if i==0:
                path = os.path.join(self.test_path, 'feat_test', filename.split(".")[0] + '.npy')
                print(path)
              else:
                path = os.path.join(self.test_path, 'feat_test', filename.split(".")[0] + "_"+  str(i+1)+ '.npy')
              
              if os.path.exists(path):
                  f = np.load(path)
                  featList.append(f)
            #print(featList)
            #print("size of featList " +  str(len(featList)))
            X_test.append(featList)
            #print("size of xtest " +  str(len(X_test)))
            X_test_filename.append(filename[:-4])            
        #X_test = np.array(X_test)   
        #print( "length of xtest is " + str(len(X_test)) )
        #print(X_test)
        return X_test, X_test_filename

    def test(self):
        """
            writes the captions of all the testing videos in a text file
        """
        X_test, X_test_filename = self.get_test_data()
        #print(X_test_filename)
        #print(X_test)
        # generate inference test outputs
        with open("/content/drive/MyDrive/Dense-Video-Captioning/data/testing_data/Beam2_Stacked.txt", 'w') as file:
            #file.write(X_test_filename)
            file.write(X_test_filename[0] + ',')
            for idx in range(0,len(X_test)):
                print(idx)
                file.write(X_test_filename[idx] + ',')
                listOfSentences=[]
                for j in range(0,len(X_test[idx])):
                  if self.search_type == 'greedy':
                          feature = np.array(X_test[idx][j])
                          sentence_predicted = self.greedy_search(feature.reshape((-1, 20, 2560)))
                          listOfSentences.append(str(sentence_predicted))
                          file.write(str(listOfSentences)+ "\n")
                  else:
                          sentence_predicted = ''
                          feature = np.array(X_test[idx][j])
                          decoded_sentence = self.decode_sequence2bs(feature.reshape((-1, 20, 2560)))
                          decode_str = self.decoded_sentence_tuning(decoded_sentence)
                          for d in decode_str:
                              sentence_predicted = sentence_predicted + d + ' '
                              listOfSentences.append(str(sentence_predicted))
                              file.write(str(listOfSentences))

                      #file.write(str(listOfSentences))
                # re-init max prob
                self.max_probability = -1


In [ ]:

if __name__ == "__main__":
    video_to_text = VideoDescriptionInference(config)
    video_to_text.load_inference_models()
    video_to_text.test()


256
tokenixer
encoder
decoder
Processing video for detecting scenes 778mkceE0UQ_40_46.avi


100%|██████████| 360/360 [00:01<00:00, 336.51frames/s]


1 scenes detected!
/content/drive/MyDrive/Dense-Video-Captioning/data/testing_data/feat_test/778mkceE0UQ_40_46.npy
Processing video for detecting scenes lw7pTwpx0K0_38_48.avi


100%|██████████| 300/300 [00:03<00:00, 80.73frames/s]


2 scenes detected!
/content/drive/MyDrive/Dense-Video-Captioning/data/testing_data/feat_test/lw7pTwpx0K0_38_48.npy
Processing video for detecting scenes sZf3VDsdDPM_107_114.avi


...

1 scenes detected!
/content/drive/MyDrive/Dense-Video-Captioning/data/testing_data/feat_test/s1ZABV7AQdA_38_48.npy
Processing video for detecting scenes pW9DFPqoIsI_26_50.avi


100%|██████████| 601/601 [00:01<00:00, 357.83frames/s]


7 scenes detected!
/content/drive/MyDrive/Dense-Video-Captioning/data/testing_data/feat_test/pW9DFPqoIsI_26_50.npy
Processing video for detecting scenes NUYu9c9XsgY_7_21.avi


100%|██████████| 211/211 [00:00<00:00, 499.09frames/s]


1 scenes detected!
/content/drive/MyDrive/Dense-Video-Captioning/data/testing_data/feat_test/NUYu9c9XsgY_7_21.npy
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66


In [ ]:
test_result={}
with open("/content/drive/MyDrive/Video-Captioning-main/data/testing_data/test_greedy.txt","r") as test_file:    
    for line in test_file.readlines():
      y_data = line.split(",")
      test_result[y_data[0]] = str(y_data[1])

print(test_result)
#nltk.translate.bleu_score.sentence_bleu(references, hypothesis)

FileNotFoundError: ignored

In [ ]:
test_label={}
count=0
with open("/content/drive/MyDrive/Video-Captioning-main/data/testing_data/testing_label.txt","r") as test_file:    
    for line in test_file.readlines():
      count+=1
      y_dataID = line.split(" ")[0]
      y_dataVal = line.partition(' ')[2]
      #print(str(y_dataID) + str(y_dataVal))
      if str(y_dataID) in test_result:
        if str(y_dataID) in test_label:
          test_label[y_dataID]=(str(y_dataVal))
          #print(test_label[y_dataID])
        else:
          if count==1:
            count+=1
            continue
          else:
            test_label[y_dataID] = (str(y_dataVal))
          #print(test_label[y_dataID])
      
      else:
        continue

print(test_label)


In [ ]:
print((y_dataVal).split(" "))

In [ ]:
print(test_label)


In [ ]:

with open("/content/drive/MyDrive/Video-Captioning-main/data/testing_data/Finaltesting_label2.txt","w") as test_file:
  for key, value in test_label.items(): 
    test_file.write('%s:%s\n' % (key, value))   
   

In [ ]:

with open("/content/drive/MyDrive/Video-Captioning-main/data/testing_data/FinalTestGreedy.txt","w") as test_file:
  for i in sorted (test_result) :
    test_file.write('%s:%s\n' % (i, test_result[i]))

In [ ]:
index=0
BLEUlist=[]
for key1,val1 in test_label.items():
  ref= val1
  hypo = test_result[key1]
  index+=1
  BLEUlist.append(nltk.translate.bleu_score.sentence_bleu(ref, hypo,weights = [0,1,0,0]))
  print(str(index) +" " + str(key1) +"  "+ str(nltk.translate.bleu_score.sentence_bleu(ref, hypo, weights = [0,1,.25,0])))

In [ ]:
print(max(BLEUlist))
print(min(BLEUlist))
print(sum(BLEUlist)/len(BLEUlist))

In [ ]:
#!pip install -U nltk
from nltk.translate.meteor_score import meteor_score
index=0
METEORlist=[]

for key1,val1 in test_label.items():
  hypo=[]
  hypo .append(test_result[key1])
  ref=[]
  ref.append(l[0])
  print
  METEORlist.append(meteor_score(['a woman cuts an oion'], ['a woman is mixing something']))
  print(str(index) +" " + str(key1) +"  "+ str(meteor_score(['a woman cuts an oion'], ['a woman is mixing something'],2)))

In [ ]:
print (nltk.translate.meteor_score.meteor_score(
    [["a woman cuts an oion"]], ["a woman is mixing something"]))

In [ ]:
mylist = [['a', 'woman', 'cuts', 'an', 'oion'], ['a', 'girl', 'is', 'cutting', 'an', 'onion'], ['a', 'girl', 'is', 'cutting', 'an', 'onion'], ['a', 'girl', 'is', 'cutting', 'onion', 'into', 'small', 'pieces'], ['a', 'girl', 'is', 'dicing', 'up', 'an', 'onion'], ['a', 'woman', 'chops', 'a', 'white', 'onion'], ['a', 'woman', 'cuts', 'and', 'dices', 'an', 'onion'], ['a', 'woman', 'is', 'chopping', 'an', 'onion', 'into', 'small', 'pieces'], ['a', 'woman', 'is', 'chopping', 'an', 'onion', 'slice', 'into', 'fine', 'pieces'], ['a', 'woman', 'is', 'cutting', 'onions'], ['a', 'woman', 'is', 'slicing', 'a', 'onion'], ['a', 'woman', 'is', 'slicing', 'some', 'vegetables'], ['a', 'woman', 'slices', 'an', 'onion'], ['an', 'onion', 'is', 'chopped', 'up'], ['the', 'cook', 'is', 'dicing', 'onions'], ['the', 'girl', 'is', 'slicing', 'an', 'onion'], ['a', 'woman', 'is', 'cutting', 'the', 'tomato', 'into', 'pieces'], ['a', 'woman', 'is', 'slicing', 'an', 'onion'], ['a', 'young', 'woman', 'cutting', 'an', 'onion', 'to', 'make', 'salsa'], ['a', 'girl', 'is', 'chopping', 'onions'], ['a', 'girl', 'is', 'slicing', 'an', 'onion'], ['a', 'lady', 'is', 'chopping', 'onions'], ['a', 'woman', 'cuts', 'an', 'onion'], ['a', 'woman', 'cutting', 'an', 'onion'], ['a', 'woman', 'in', 'a', 'red', 'dress', 'chopping', 'tomatoes'], ['a', 'woman', 'is', 'chopping', 'onions'], ['a', 'woman', 'is', 'chopping', 'onions'], ['a', 'woman', 'is', 'chopping', 'up', 'onions'], ['a', 'woman', 'is', 'cutting', 'an', 'onion'], ['a', 'woman', 'is', 'cutting', 'onions'], ['a', 'woman', 'is', 'dicing', 'an', 'onion'], ['a', 'woman', 'is', 'dicing', 'an', 'onion'], ['a', 'woman', 'is', 'slicing', 'a', 'salsa'], ['a', 'woman', 'is', 'slicing', 'an', 'onion'], ['a', 'woman', 'is', 'slicing', 'an', 'onion'], ['a', 'woman', 'is', 'slicing', 'and', 'chopping', 'up', 'an', 'onion'], ['a', 'woman', 'is', 'slicing', 'and', 'dicing', 'an', 'onion'], ['a', 'woman', 'is', 'slicing', 'some', 'tomatoes'], ['a', 'woman', 'is', 'slicing', 'some', 'vegetables'], ['a', 'woman', 'is', 'slicing', 'some', 'vegitable'], ['a', 'woman', 'is', 'slicing', 'then', 'dicing', 'an', 'onion'], ['a', 'woman', 'is', 'slicing', 'vegetables'], ['the', 'girl', 'sliced', 'an', 'onion'], ['the', 'girl', 'sliced', 'an', 'onion'], ['the', 'lady', 'is', 'slicing', 'onions'], ['the', 'woman', 'is', 'dicing', 'an', 'onion'], ['woman', 'teaches', 'on', 'how', 'to', 'cook', 'salsa'], ['a', 'lady', 'is', 'cutting', 'onion', 'with', 'the', 'knife']]
l=[]
for i in mylist:
  s=[]
  s.append(" ".join(i))
  l.append(s)
print(l)

In [ ]:
print(max(METEORlist))
print(min(METEORlist))
print(sum(METEORlist)/len(METEORlist))